### Importar base de dados e instalar framework de recomendação

### Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np

### Explorar Metados

In [ ]:
metadata = pd.read_csv('AmazonMusic/amazon_music_metadata.csv')
metadata.head()

In [ ]:
metadata.shape[0]

In [ ]:
df_meta = metadata.iloc[:, 1:]
df_meta = df_meta.melt(id_vars=['title'])
df_meta = df_meta[df_meta.value == 1]
df_meta.head()

In [ ]:
dict_title = np.load('dict_title.npy', allow_pickle=True).tolist()
inverse_dict_title = {value: int(key) for key, value in dict_title.items()}

In [ ]:
df_meta['id'] = df_meta['title'].map(inverse_dict_title)

In [ ]:
len(inverse_dict_title)

In [ ]:
df_meta.dropna(inplace=True)
df_meta['id'] = df_meta.id.astype(int)
df_meta.rename(columns={'variable': 'genre'}, inplace=True)
# df_meta = df_meta[['id', 'title', 'variable', 'value']]
df_meta = df_meta[['id', 'genre', 'value']]
df_meta.sort_values('id', inplace=True)
df_meta.head()

In [ ]:
df_meta.to_csv('AmazonMusic/metadata_file.csv', index=False, sep='\t', header=False)

In [ ]:
! ls AmazonMusic/

### Recomendação

In [ ]:
from caserec.recommenders.item_recommendation.item_attribute_knn import ItemAttributeKNN

In [ ]:
iknn = ItemAttributeKNN('train.txt', 'test.txt', metadata_file='AmazonMusic/metadata_file.csv', 
                        as_similar_first=False)
iknn.compute(as_table=True, metrics=['Prec'])

In [ ]:
np.mean(iknn.si_matrix), np.std(iknn.si_matrix)

### Criando arquivo de similaridade entre os metadados

In [ ]:
n_matrix = iknn.si_matrix.copy()
n_matrix[n_matrix<0.2] = 0

In [ ]:
np.mean(n_matrix), np.std(n_matrix)

In [ ]:
df_matrix = pd.DataFrame(n_matrix)
df_matrix = df_matrix.stack()
df_matrix = df_matrix.reset_index()
# A biblioteca permite que vc passe so metadados diferentes de zero, assim o arquivo fica menor
df_matrix = df_matrix[df_matrix[0] != 0]
df_matrix.head(3)

In [ ]:
df_matrix.to_csv('AmazonMusic/new_similarity_matrix.csv', index=False, sep='\t', header=False)

In [ ]:
iknn = ItemAttributeKNN('train.txt', 'test.txt', similarity_file='AmazonMusic/new_similarity_matrix.csv',
                       as_similar_first=False)
iknn.compute(as_table=True, metrics=['Prec'])